<a href="https://colab.research.google.com/github/Rob174/Astronomy/blob/Astronomy/Modele000002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

##Python / Colab
from google.colab import files
from google.colab import drive
import os
from IPython.display import Image as imgIPython
from IPython.display import clear_output,display
import IPython
## Tensorflow keras
try:
  !pip install -q tf-nightly
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.python import debug as tf_debug
from tensorflow.python.client import device_lib

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense,Conv2D,Convolution2D,Activation,Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D,AveragePooling2D
from tensorflow.keras.layers import Dropout,Reshape,BatchNormalization
from tensorflow.keras.layers import concatenate,Concatenate,Subtract,Multiply,Average,Add
from tensorflow.keras.layers import UpSampling2D, Reshape,Flatten
from tensorflow.keras.layers import Lambda

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import models
from tensorflow.keras.models import Model
import tensorflow.keras.losses
## Math libraries
import numpy as np
import scipy
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
##Images
from PIL import Image
import cv2
## Graph
from graphviz import render
from graphviz import Digraph,Graph
drive.mount('/content/drive')
%cd '/content/drive/My Drive/TIPE'
#Dataset
from __future__ import absolute_import, division, print_function, unicode_literals
import random
import pathlib
import shutil
import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TIPE


#Dataset

In [0]:
#@title { display-mode: "form" }

dossier_TIPE = "/content/drive/My Drive/TIPE/"#@param {type:"string"}

Vérification des datasets

In [3]:
path = dossier_TIPE+"Galaxies_resized/"
dossiers = ["Train","Validation","Test"]
compte = []
for d in dossiers:
    compte.append(len(os.listdir(path+d)))
print([c/sum(compte) for c in compte])
print([c/10 for c in compte])

[0.6089108910891089, 0.297029702970297, 0.09405940594059406]
[24.6, 12.0, 3.8]


Création des classes de dataset Tensorflow

In [0]:
#D'après https://stackoverflow.com/questions/54590363/create-tensorflow-dataset-from-image-local-directory
class ArtificialDataset(tf.data.Dataset):
    #Bruit gaussien origine : https://stackoverflow.com/questions/59286171/gaussian-blur-image-in-dataset-pipeline-in-tensorflow
    _COMPTEUR = 0
    def _gaussian_kernel(kernel_size, sigma, n_channels, dtype):
        """Génère le noyau de la couche de convolution du flou gaussien"""
        x = tf.range(-kernel_size // 2 + 1, kernel_size // 2 + 1, dtype=dtype)
        g = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(tf.cast(sigma, dtype), 2))))
        g_norm2d = tf.pow(tf.reduce_sum(g), 2)
        g_kernel = tf.tensordot(g, g, axes=0) / g_norm2d
        g_kernel = tf.expand_dims(g_kernel, axis=-1)
        return tf.expand_dims(tf.tile(g_kernel, (1, 1, n_channels)), axis=-1)

    def apply_blur(img):
        """Applique le flou gaussien"""
        img = tf.reshape(img,[1,256,256,3])
        blur = _gaussian_kernel(3, 2, 3, img.dtype)
        img = tf.nn.depthwise_conv2d(img, blur, [1,1,1,1], 'SAME')
        return tf.reshape(img,[256,256,3])

    def open_imgs(x):
        """Modification par image"""
        img = tf.io.read_file(x)
        img = tf.image.decode_jpeg(img)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, [256, 256])
        img = tf.clip_by_value(img,0,1)
        #Bruitage
        img_noise = tf.image.adjust_saturation(img,.8)
        img_noise = apply_blur(img_noise)
        img_noise = tf.image.adjust_contrast(img_noise,tf.reduce_sum(tf.random.uniform(shape=(1,),dtype=tf.float32,minval=0.1,maxval=0.2)))
        ajout = 0.1
        img_noise = img_noise+tf.concat([tf.ones(shape=[256,256,1])*ajout,tf.zeros(shape=[256,256,2])],axis=-1)
        img_noise = tf.image.adjust_brightness(img_noise,-ajout)
        img_noise = tf.image.adjust_hue(img_noise,tf.reduce_sum(tf.random.uniform(shape=(1,),dtype=tf.float32,minval=0.06,maxval=0.09)))
        img_noise = tf.clip_by_value(img_noise,0,1)
        return img,img_noise
        
    def _generator():
        for img_batch in ArtificialDataset.donnees:
            refs,entrees = [],[]
            for i,img in enumerate(img_batch):
                clean,noise = ArtificialDataset.open_imgs(img)
                refs.append(clean)
                if i < 5:
                    entrees.append(noise)
                else:
                    entrees.append(clean)
            refs = tf.stack(refs)
            entrees = tf.stack(entrees)
            yield (refs,entrees)
    
    def __new__(cls,nom):
        ArtificialDataset.donnees = tf.data.Dataset.list_files(dossier_TIPE+"Galaxies_resized/"+nom+"/*.jpg").shuffle(len(os.listdir(dossier_TIPE+"Galaxies_resized/"+nom))).batch(10)
        #voir doc from_generator https://github.com/tensorflow/tensorflow/blob/f2b2563c6ce2001a117cd7adb36f303903e907ec/tensorflow/python/data/ops/dataset_ops.py#L669
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float32,tf.dtypes.float32),
            output_shapes=(tf.TensorShape([None,256,256,3]), tf.TensorShape([None,256,256,3])), #https://www.tensorflow.org/api_docs/python/tf/TensorShape?version=nightly
            args=None
        )


Test de la classe

In [0]:
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.05)
    tf.print("Execution time:", time.perf_counter() - start_time)

In [0]:
benchmark(ArtificialDataset(nom="Train").prefetch(7))